In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from typing import List

'''
None || 2000
'''
nb = None
nb_trains = nb
nb_valids = nb
nb_tests = None

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
cd drive/MyDrive/Projects/JPX_Tokyo_Stock/JPX_stock_2022/working

/content/drive/MyDrive/Projects/JPX_Tokyo_Stock/JPX_stock_2022/working


In [4]:
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import optuna
from data_utils import *

In [6]:
train_df = pd.read_csv('../__input__/1_DataPreprocessing/train_df.csv', parse_dates=['Date'], nrows = nb_trains)
valid_df = pd.read_csv('../__input__/1_DataPreprocessing/valid_df.csv', parse_dates=['Date'], nrows = nb_trains)
test_df = pd.read_csv('../__input__/1_DataPreprocessing/test_df.csv', parse_dates=['Date'], nrows = nb_trains)

### check info

In [7]:
train_df.info(null_counts= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1863182 entries, 0 to 1863181
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Date              1863182 non-null  datetime64[ns]
 1   SecuritiesCode    1863182 non-null  int64         
 2   ExpectedDividend  1863182 non-null  float64       
 3   AdjustmentFactor  1863182 non-null  float64       
 4   SupervisionFlag   1863182 non-null  bool          
 5   Volume            1863182 non-null  int64         
 6   Open              1863182 non-null  float64       
 7   High              1863182 non-null  float64       
 8   Low               1863182 non-null  float64       
 9   Close             1863182 non-null  float64       
 10  Target            1863182 non-null  float64       
 11  CloseT1           1863182 non-null  float64       
 12  CloseT2           1863182 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(9),

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [8]:
train_df.head(20)

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
0,2017-01-04,1301,-1.0,1.0,False,31400,2734.0,2755.0,2730.0,2742.0,0.000730,2738.0,2740.0
1,2017-01-04,1332,-1.0,1.0,False,2798500,568.0,576.0,563.0,571.0,0.012324,568.0,575.0
2,2017-01-04,1333,-1.0,1.0,False,270800,3150.0,3210.0,3140.0,3210.0,0.006154,3250.0,3270.0
3,2017-01-04,1376,-1.0,1.0,False,11300,1510.0,1550.0,1510.0,1550.0,0.011053,1538.0,1555.0
4,2017-01-04,1377,-1.0,1.0,False,150800,3270.0,3350.0,3270.0,3330.0,0.003026,3305.0,3315.0
5,2017-01-04,1379,-1.0,1.0,False,77300,2105.0,2147.0,2101.0,2143.0,0.005169,2128.0,2139.0
6,2017-01-04,1381,-1.0,1.0,False,1300,1950.0,1960.0,1949.0,1960.0,-0.009326,1930.0,1912.0
7,2017-01-04,1407,-1.0,1.0,False,147000,857.0,877.0,851.0,866.0,-0.003437,885.0,882.0
8,2017-01-04,1414,-1.0,1.0,False,119600,4940.0,5060.0,4935.0,5050.0,0.000000,5030.0,5030.0
9,2017-01-04,1417,-1.0,1.0,False,347500,1051.0,1063.0,1048.0,1053.0,-0.007463,1072.0,1064.0


In [9]:
valid_df.info(null_counts= True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232663 entries, 0 to 232662
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              232663 non-null  datetime64[ns]
 1   SecuritiesCode    232663 non-null  int64         
 2   ExpectedDividend  232663 non-null  float64       
 3   AdjustmentFactor  232663 non-null  float64       
 4   SupervisionFlag   232663 non-null  bool          
 5   Volume            232663 non-null  int64         
 6   Open              232663 non-null  float64       
 7   High              232663 non-null  float64       
 8   Low               232663 non-null  float64       
 9   Close             232663 non-null  float64       
 10  Target            232663 non-null  float64       
 11  CloseT1           232663 non-null  float64       
 12  CloseT2           232663 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(9), int64(2)
memory 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [10]:
valid_df.head(20)

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
0,2021-05-28,1301,-1.0,1.0,False,96000,2905.0,2909.0,2851.0,2869.0,0.026934,2896.0,2974.0
1,2021-05-28,1332,-1.0,1.0,False,1578600,509.0,517.0,508.0,514.0,0.013807,507.0,514.0
2,2021-05-28,1333,-1.0,1.0,False,148900,2405.0,2420.0,2400.0,2415.0,0.016701,2395.0,2435.0
3,2021-05-28,1375,-1.0,1.0,False,156900,1781.0,1797.0,1764.0,1770.0,-0.006384,1723.0,1712.0
4,2021-05-28,1376,-1.0,1.0,False,8300,1540.0,1540.0,1506.0,1524.0,0.006098,1476.0,1485.0
5,2021-05-28,1377,-1.0,1.0,False,424500,3645.0,3720.0,3630.0,3680.0,0.005384,3715.0,3735.0
6,2021-05-28,1379,-1.0,1.0,False,105000,1948.0,1953.0,1936.0,1941.0,0.002069,1933.0,1937.0
7,2021-05-28,1381,-1.0,1.0,False,1700,3420.0,3430.0,3405.0,3420.0,-0.005857,3415.0,3395.0
8,2021-05-28,1407,-1.0,1.0,False,234900,3485.0,3540.0,3440.0,3540.0,-0.022599,3540.0,3460.0
9,2021-05-28,1413,-1.0,1.0,False,20200,2512.0,2522.0,2484.0,2512.0,0.009604,2499.0,2523.0


In [11]:
test_df.info(null_counts= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198663 entries, 0 to 198662
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              198663 non-null  datetime64[ns]
 1   SecuritiesCode    198663 non-null  int64         
 2   ExpectedDividend  198663 non-null  float64       
 3   AdjustmentFactor  198663 non-null  float64       
 4   SupervisionFlag   198663 non-null  bool          
 5   Volume            198663 non-null  int64         
 6   Open              198663 non-null  float64       
 7   High              198663 non-null  float64       
 8   Low               198663 non-null  float64       
 9   Close             198663 non-null  float64       
 10  Target            198663 non-null  float64       
 11  CloseT1           198663 non-null  float64       
 12  CloseT2           198663 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(9), int64(2)
memory 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [12]:
test_df.head(20)

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
0,2021-12-06,1301,-1.0,1.0,False,8900,2982.0,2982.0,2965.0,2971.0,-0.003263,3065.0,3055.0
1,2021-12-06,1332,-1.0,1.0,False,1360800,592.0,599.0,588.0,589.0,-0.008993,556.0,551.0
2,2021-12-06,1333,-1.0,1.0,False,125900,2368.0,2388.0,2360.0,2377.0,-0.009963,2409.0,2385.0
3,2021-12-06,1375,-1.0,1.0,False,81100,1230.0,1239.0,1224.0,1224.0,-0.015032,1264.0,1245.0
4,2021-12-06,1376,-1.0,1.0,False,6200,1339.0,1372.0,1339.0,1351.0,0.002867,1395.0,1399.0
5,2021-12-06,1377,-1.0,1.0,False,77000,3185.0,3215.0,3180.0,3200.0,0.003044,3285.0,3295.0
6,2021-12-06,1379,-1.0,1.0,False,48500,1890.0,1898.0,1885.0,1889.0,-0.009870,1925.0,1906.0
7,2021-12-06,1381,-1.0,1.0,False,1600,3200.0,3200.0,3150.0,3150.0,0.000000,3165.0,3165.0
8,2021-12-06,1407,-1.0,1.0,False,246700,6100.0,6290.0,6010.0,6220.0,-0.004732,6340.0,6310.0
9,2021-12-06,1413,-1.0,1.0,False,9200,2185.0,2211.0,2181.0,2190.0,0.002222,2250.0,2255.0


### Calc Scores

In [13]:
def calc_spread_return_per_day(df, portfolio_size: int = 200, toprank_weight_ratio: float = 2):
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): spread return
    """
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    buf = df.groupby('Date').apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio, buf

def add_rank(df: pd.DataFrame, y_pred):
    df["Pred"] = y_pred
    df["Rank"] = df.groupby("Date")["Pred"].rank(ascending=False, method="first") - 1 
    df = df.drop("Pred", axis= 1)
    return df

In [14]:
def calc_score(df: pd.DataFrame, y_pred: pd.DataFrame, y_true: pd.DataFrame, render_info= True):
    feature_df = df.copy()
    feature_df = add_rank(feature_df, y_pred)
    feature_df['Target'] = y_true
    score, buf = calc_spread_return_sharpe(feature_df)
    if render_info:
        print(f'score -> {score}\nmean -> {buf.mean()}\nstd -> {buf.std()}')
    return score

In [15]:
calc_score(test_df, test_df["Target"], test_df["Target"])

score -> 4.736669470975898
mean -> 15.115996128121248
std -> 3.1912710440838277


4.736669470975898

### Cross Validation

In [16]:
featureCols = train_df.drop(["Target", "CloseT1", "CloseT2"], axis = 1).columns
featureCols

Index(['Date', 'SecuritiesCode', 'ExpectedDividend', 'AdjustmentFactor',
       'SupervisionFlag', 'Volume', 'Open', 'High', 'Low', 'Close'],
      dtype='object')

In [17]:
X_train = train_df[featureCols]
X_train["Date"] = X_train["Date"].astype(str)
y_train = train_df["Target"]

X_valid = valid_df[featureCols]
X_valid["Date"] = X_valid["Date"].astype(str)
y_valid = valid_df["Target"]

X_test = test_df[featureCols]
X_test["Date"] = X_test["Date"].astype(str)
yTest_true = test_df["Target"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [18]:
calc_score(X_test, yTest_true, yTest_true)

score -> 4.736669470975898
mean -> 15.115996128121248
std -> 3.1912710440838277


4.736669470975898

#### Optuna tuning

In [21]:
def objective(
    trial,
    random_state=22,
    n_jobs=-1,
):
    # XGBoost parameters
    params = {
        'tree_method':'gpu_hist',
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "objective": "reg:squarederror",
        "n_estimators": trial.suggest_int("n_estimators", 1, 10) * 100,
        "max_depth": trial.suggest_int("max_depth", 4, 20),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample": trial.suggest_loguniform("subsample", 0.6, 0.8),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "seed": random_state,
        "n_jobs": n_jobs,
        "early_stopping_rounds": 50,
        "eval_metric": "rmse"
        
    }
    
    model = xgb.XGBRegressor(**params) 

    model.fit(
        X_train.drop("Date", axis = 1),
        y_train,
        eval_set=[(X_valid.drop("Date", axis = 1), y_valid)],
        verbose=0,
    )
    y_pred = pd.DataFrame(model.predict(X_valid.drop("Date", axis = 1)), columns =['Target'])
    return calc_score(X_valid, y_pred, y_valid, render_info= False)

In [ ]:
# break point
1/0

ZeroDivisionError: ignored

In [22]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-06-29 16:30:10,203] A new study created in memory with name: no-name-28af480b-4050-45fe-9298-b61486febe6f
[I 2022-06-29 16:30:30,797] Trial 0 finished with value: 0.15432092914304335 and parameters: {'n_estimators': 3, 'max_depth': 16, 'learning_rate': 0.026063240397830766, 'colsample_bytree': 0.4068903702886825, 'subsample': 0.6764831257764504, 'alpha': 0.18916435794660133, 'lambda': 0.00027495368834733105, 'min_child_weight': 79.16810838964635}. Best is trial 0 with value: 0.15432092914304335.
[I 2022-06-29 16:30:57,489] Trial 1 finished with value: 0.16737136041950887 and parameters: {'n_estimators': 7, 'max_depth': 13, 'learning_rate': 0.020932134755723687, 'colsample_bytree': 0.34659044946382245, 'subsample': 0.752533638979545, 'alpha': 6.213386815850906, 'lambda': 0.03155501452046849, 'min_child_weight': 187.42844975740292}. Best is trial 1 with value: 0.16737136041950887.
[I 2022-06-29 16:31:00,681] Trial 2 finished with value: 0.09069674183853481 and parameters: {'n_est

Number of finished trials: 30
Best trial: {'n_estimators': 5, 'max_depth': 6, 'learning_rate': 0.01681374981485902, 'colsample_bytree': 0.46800630570537544, 'subsample': 0.6612851239732956, 'alpha': 0.02116732639687475, 'lambda': 3.747006727814721e-07, 'min_child_weight': 143.42715946748118}
CPU times: user 6min 52s, sys: 9.74 s, total: 7min 2s
Wall time: 7min


In [27]:
best_params = {
  'n_estimators': 5, 
  'max_depth': 6, 
  'learning_rate': 0.01681374981485902, 
  'colsample_bytree': 0.46800630570537544, 
  'subsample': 0.6612851239732956, 
  'alpha': 0.02116732639687475, 
  'lambda': 3.747006727814721e-07, 
  'min_child_weight': 143.42715946748118,
  'early_stopping_rounds': 50,
  'eval_metric': 'rmse',
  'n_estimators': 1000,
  'n_jobs': -1,
  'objective': 'reg:squarederror',
  'seed': 22,
  'tree_method': 'gpu_hist',
  'verbosity': 0
}

best_params

{'alpha': 0.02116732639687475,
 'colsample_bytree': 0.46800630570537544,
 'early_stopping_rounds': 50,
 'eval_metric': 'rmse',
 'lambda': 3.747006727814721e-07,
 'learning_rate': 0.01681374981485902,
 'max_depth': 6,
 'min_child_weight': 143.42715946748118,
 'n_estimators': 1000,
 'n_jobs': -1,
 'objective': 'reg:squarederror',
 'seed': 22,
 'subsample': 0.6612851239732956,
 'tree_method': 'gpu_hist',
 'verbosity': 0}

In [28]:
best_params = study.best_trial.params
best_params['tree_method'] = 'gpu_hist'
best_params['verbosity'] = 0
best_params['objective'] = "reg:squarederror"
best_params['seed'] = 22
best_params['n_jobs'] = -1
best_params["early_stopping_rounds"] = 50
best_params["eval_metric"] = "rmse"

best_params

{'alpha': 0.02116732639687475,
 'colsample_bytree': 0.46800630570537544,
 'early_stopping_rounds': 50,
 'eval_metric': 'rmse',
 'lambda': 3.747006727814721e-07,
 'learning_rate': 0.01681374981485902,
 'max_depth': 6,
 'min_child_weight': 143.42715946748118,
 'n_estimators': 5,
 'n_jobs': -1,
 'objective': 'reg:squarederror',
 'seed': 22,
 'subsample': 0.6612851239732956,
 'tree_method': 'gpu_hist',
 'verbosity': 0}

In [29]:
%%time
model = xgb.XGBRegressor(**best_params) 

model.fit(
    X_train.drop("Date", axis = 1),
    y_train,
    eval_set=[(X_valid.drop("Date", axis = 1), y_valid)],
    verbose=0,
)

CPU times: user 1.86 s, sys: 301 ms, total: 2.16 s
Wall time: 2.14 s


In [30]:
%%time
y_pred = pd.DataFrame(model.predict(X_train.drop("Date", axis = 1)), columns =['Target'])
print(calc_score(X_train, y_pred, y_train, render_info= False))

0.10009867027947529
CPU times: user 3.26 s, sys: 264 ms, total: 3.53 s
Wall time: 3.5 s


In [31]:
np.sqrt(mean_squared_error(y_pred, y_train))

0.459679275214446

In [32]:
%%time
y_pred = pd.DataFrame(model.predict(X_valid.drop("Date", axis = 1)), columns =['Target'])
print(calc_score(X_valid, y_pred, y_valid, render_info= False))

0.11558036193766377
CPU times: user 416 ms, sys: 18 ms, total: 434 ms
Wall time: 429 ms


In [33]:
np.sqrt(mean_squared_error(y_pred, y_valid))

0.45965080818393295

In [35]:
%%time
y_pred = pd.DataFrame(model.predict(X_test.drop("Date", axis = 1)), columns =['Target'])
print(calc_score(X_test, y_pred, yTest_true, render_info= False))

0.05491410986293702
CPU times: user 356 ms, sys: 12 ms, total: 368 ms
Wall time: 364 ms


In [37]:
np.sqrt(mean_squared_error(y_pred, yTest_true))

0.4607718562986889

#### test pred hidden_test

In [42]:
hidden_test_df = pd.read_csv('../__input__/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv', nrows = None, parse_dates=['Date'])
mini_valid = pd.read_csv('../__input__/1_DataPreprocessing/mini_valid.csv', parse_dates=['Date'], nrows = None)

In [39]:
hidden_test_df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False
1,20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False
2,20211206_1333,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False
3,20211206_1375,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False
4,20211206_1376,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
3995,20211207_9990,2021-12-07,9990,526.0,535.0,524.0,535.0,57800,1.0,NaN,False
3996,20211207_9991,2021-12-07,9991,795.0,806.0,792.0,805.0,48500,1.0,NaN,False
3997,20211207_9993,2021-12-07,9993,1640.0,1640.0,1620.0,1620.0,6600,1.0,NaN,False
3998,20211207_9994,2021-12-07,9994,2437.0,2440.0,2423.0,2440.0,5200,1.0,NaN,False


In [43]:
y_test_official = DataPreprocessing_for_HiddenTest(hidden_test_df, mini_valid)
y_test_official

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,-1.0,False
1,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,-1.0,False
2,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,-1.0,False
3,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,-1.0,False
4,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,-1.0,False
...,...,...,...,...,...,...,...,...,...,...
3995,2021-12-07,9990,526.0,535.0,524.0,535.0,57800,1.0,-1.0,False
3996,2021-12-07,9991,795.0,806.0,792.0,805.0,48500,1.0,-1.0,False
3997,2021-12-07,9993,1640.0,1640.0,1620.0,1620.0,6600,1.0,-1.0,False
3998,2021-12-07,9994,2437.0,2440.0,2423.0,2440.0,5200,1.0,-1.0,False


### Submission

In [ ]:
# import jpx_tokyo_market_prediction
# env = jpx_tokyo_market_prediction.make_env()
# iter_test = env.iter_test()
# for (df_test, options, financials, trades, secondary_prices, df_pred) in iter_test:
#     print(df_test.info())
#     print(df_test.head(10))
#     print(secondary_prices.head(10))
    
    
#     env_x_test = EDA_df_for_hidden_test(df_test, train_df)

#     env_y_pred =  pd.DataFrame(model.predict(env_x_test.drop("Date", axis = 1)), columns =['Target'])
    
#     df_pred['Target'] = env_y_pred
#     df_pred = add_rank(df_pred, env_y_pred)
    
#     submission = df_pred[["Date", "SecuritiesCode", "Rank"]]  
#     print(submission.columns)
#     env.predict(submission)